In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from analysis import *
from getters import * 

{'w': Parameter containing:
tensor([[-0.1910, -0.1702, -0.1453,  0.0324,  0.1322,  0.0163, -0.0556, -0.0668,
          0.0829,  0.1951,  0.1547, -0.1675,  0.1080,  0.0485,  0.1479,  0.1093,
         -0.0553, -0.0663, -0.0140,  0.0825, -0.1408,  0.1124,  0.0499, -0.1040,
         -0.1081, -0.0461, -0.0448,  0.0271, -0.0626,  0.0957, -0.1187, -0.0656,
         -0.0442,  0.0033,  0.1118]], requires_grad=True)}
{'w': Parameter containing:
tensor([[-0.1910, -0.1702, -0.1453,  0.0324,  0.1322,  0.0163, -0.0556, -0.0668,
          0.0829,  0.1951,  0.1547, -0.1675,  0.1080,  0.0485,  0.1479,  0.1093,
         -0.0553, -0.0663, -0.0140,  0.0825, -0.1408,  0.1124,  0.0499, -0.1040,
         -0.1081, -0.0461, -0.0448,  0.0271, -0.0626,  0.0957, -0.1187, -0.0656,
         -0.0442,  0.0033,  0.1118]], requires_grad=True)}
[9, 0, 1, 11, 10]
[9, 0, 1, 11, 10]
1.0
[9, 0, 1, 11, 10, 14, 2, 20, 4, 30]
[9, 0, 1, 11, 10, 14, 2, 20, 4, 30]
1.0
[9, 0, 1, 11, 10, 14, 2, 20, 4, 30, 21, 34, 15, 24, 12]
[9, 0,

In [5]:
views=[0, 2, 4, 5]
models=["gcn", "gcn_student", "gcn_student_teacher", "gcn_student_teacher_weight"]
CV=["3Fold", "5Fold", "10Fold"]
run=1
dataset = "gender_data"

view_data_mean = []
view_data_std = []

for view in views:
    
    barWidth = 1/(len(models)+1)
    model_result_mean = []
    model_result_std = []
    
    for model in models:
        rep_score, std = view_specific_rep(dataset=dataset, view=view, model=model, run=run, CV=CV)
        model_result_mean.append(rep_score)
        model_result_std.append(std)
    
    view_data_mean.append(model_result_mean)
    view_data_std.append(model_result_std)

view_data_std.append(list(np.std(view_data_mean, axis=0)))
view_data_std = np.array(view_data_std).T

view_data_mean.append(list(np.mean(view_data_mean, axis=0)))
view_data_mean = np.array(view_data_mean).T


In [4]:
pd.DataFrame(view_data_mean, columns=["View {}".format(i) for i in views]+ ["Average"], index=["Teacher", "Student", "Student + Teacher", "Student + Teacher + Weight Loss"])

,View 0,View 2,View 4,View 5,Average
Teacher,0.426389,0.477778,0.511111,0.444444,0.464931
Student,0.376389,0.411111,0.433333,0.634722,0.463889
Student + Teacher,0.411111,0.423611,0.473611,0.640278,0.487153
Student + Teacher + Weight Loss,0.440278,0.579167,0.561111,0.444444,0.506250


In [6]:
pd.DataFrame(view_data_std, columns=["View {}".format(i) for i in views]+ ["Average"], index=["Teacher", "Student", "Student + Teacher", "Student + Teacher + Weight Loss"])

,View 0,View 2,View 4,View 5,Average
Teacher,0.137535,0.113363,0.060508,0.045938,0.032415
Student,0.034415,0.085549,0.051031,0.036694,0.100697
Student + Teacher,0.048153,0.059932,0.061080,0.079810,0.091447
Student + Teacher + Weight Loss,0.072196,0.012266,0.087489,0.021606,0.064224


In [44]:
# Analysis for each view we create the metric tables across the different models 

In [8]:
def metric_and_view_analysis(models, CV, analysis_type, view, run, dataset_split, metric):

    all_data_mean = []
    all_data_std = []
    
    for model in models:
        
        model_results_mean = []
        model_results_std = []
       
        for training_type in CV:
            metrics = extract_metrics(dataset=dataset, model=model, analysis_type=analysis_type, training_type=training_type, view=view, run=run, dataset_split=dataset_split, metric=metric)
            mean = np.mean([metric[-1] for metric in metrics])
            std = np.std([metric[-1] for metric in metrics])
            model_results_mean.append(mean)
            model_results_std.append(std)
        
        all_data_mean.append(model_results_mean)
        all_data_std.append(model_results_std)
    
    return all_data_mean, all_data_std
        

In [10]:
views=[0, 2, 4, 5]
models=["gcn", "gcn_student", "gcn_student_teacher", "gcn_student_teacher_weight"]
CV=["3Fold", "5Fold", "10Fold"]
run=1
dataset = "gender_data"
dataset_split = ["val", "train", "test"]
metrics = ["acc", "f1", "recall", "precision"]

for view in views:
    for metric in metrics:
        print("____________________________________________________")
        print(f"View: {view} Metric: {metric}")
        mean, std = metric_and_view_analysis(models=models, 
                                            CV=CV, 
                                            analysis_type="model_assessment", 
                                            view=view, 
                                            run=run, 
                                            dataset_split="val", 
                                            metric=metric)
        mean = pd.DataFrame(mean, columns=CV, index=["Teacher", "Student", "Student + Teacher", "Student + Teacher + Weight Loss"]).round(3)
        std = pd.DataFrame(std, columns=CV, index=["Teacher", "Student", "Student + Teacher", "Student + Teacher + Weight Loss"]).round(3)
        print(mean.astype(str) + u"\u00B1" + std.astype(str))

____________________________________________________
View: 0 Metric: acc
                                       3Fold        5Fold       10Fold
Teacher                          0.602±0.017  0.593±0.038  0.599±0.057
Student                          0.571±0.049  0.603±0.042  0.575±0.042
Student + Teacher                0.582±0.042  0.602±0.042  0.592±0.042
Student + Teacher + Weight Loss   0.532±0.01    0.54±0.04   0.56±0.058
____________________________________________________
View: 0 Metric: f1
                                       3Fold        5Fold       10Fold
Teacher                          0.437±0.046   0.487±0.04  0.519±0.068
Student                          0.381±0.087   0.467±0.08  0.423±0.095
Student + Teacher                  0.35±0.14  0.492±0.071   0.436±0.13
Student + Teacher + Weight Loss   0.17±0.094  0.339±0.077  0.311±0.167
____________________________________________________
View: 0 Metric: recall
                                       3Fold        5Fold       10Fol